# The New York Social Graph


## [New York Social Diary](https://web.archive.org/web/20150913224145/http://www.newyorksocialdiary.com/) provides a
fascinating lens onto New York's socially well-to-do.  The data forms a natural social graph for New York's social elite.  Take a look at this page of a recent [run-of-the-mill holiday party](https://web.archive.org/web/20150913224145/http://www.newyorksocialdiary.com/party-pictures/2014/holiday-dinners-and-doers). Please note that these links point to the internet archive, as the original website has recently removed most of its archives. Many of the images no longer load, but all the HTML is still there.

Besides the brand-name celebrities, you will notice the photos have carefully annotated captions labeling those that appear in the photos.  We can think of this as implicitly implying a social graph: there is a connection between two individuals if they appear in a picture together.

For this project, we will assemble the social graph from photo captions for parties dated December 1, 2014, and before.  Using this graph, we can make guesses at the most popular socialites, the most influential people, and the most tightly coupled pairs.

We will attack the project in three phases:
1. Get a list of all the photo pages to be analyzed.
2. Parse all of the captions on a sample page.
3. Parse all of the captions on all pages, and assemble the graph.

Good Luck with the course!!

## Question 1: histogram


Get the number of party pages for the 95 months (that is, month-year pair) in the data.  Represent this histogram as a list of 95 tuples, each of the form `("Dec-2014", 1)`.  Note that you can convert `datetime` objects into these sort of strings with `strftime` and the [format codes for dates](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior).

Plot the histogram for yourself.  Do you see any trends?

In [559]:
histogram

(('Jan-2008', 9),
 ('Jan-2009', 8),
 ('Jan-2010', 12),
 ('Jan-2011', 11),
 ('Jan-2012', 8),
 ('Jan-2013', 4),
 ('Jan-2014', 8),
 ('Feb-2007', 1),
 ('Feb-2008', 10),
 ('Feb-2009', 10),
 ('Feb-2010', 16),
 ('Feb-2011', 11),
 ('Feb-2012', 9),
 ('Feb-2013', 10),
 ('Feb-2014', 8),
 ('Mar-2007', 18),
 ('Mar-2008', 17),
 ('Mar-2009', 16),
 ('Mar-2010', 17),
 ('Mar-2011', 15),
 ('Mar-2012', 10),
 ('Mar-2013', 11),
 ('Mar-2014', 11),
 ('Apr-2007', 17),
 ('Apr-2008', 15),
 ('Apr-2009', 15),
 ('Apr-2010', 15),
 ('Apr-2011', 15),
 ('Apr-2012', 12),
 ('Apr-2013', 12),
 ('Apr-2014', 8),
 ('May-2007', 20),
 ('May-2008', 17),
 ('May-2009', 13),
 ('May-2010', 18),
 ('May-2011', 16),
 ('May-2012', 14),
 ('May-2013', 14),
 ('May-2014', 12),
 ('June-2007', 10),
 ('June-2008', 15),
 ('June-2009', 16),
 ('June-2010', 19),
 ('June-2011', 14),
 ('June-2012', 14),
 ('June-2013', 13),
 ('June-2014', 10),
 ('July-2007', 15),
 ('July-2008', 16),
 ('July-2009', 18),
 ('July-2010', 18),
 ('July-2011', 14),
 ('July-

In [560]:
grader.score('graph__histogram', histogram)

Your score:  0.9789473684210537


## Phase Two


In this phase, we concentrate on getting the names out of captions for a given page.  We'll start with [the benefit cocktails and dinner](https://web.archive.org/web/20150913224145/http://www.newyorksocialdiary.com/party-pictures/2015/celebrating-the-neighborhood) for [Lenox Hill Neighborhood House](http://www.lenoxhill.org/), a neighborhood organization for the East Side.

Take a look at that page.  Note that some of the text on the page is captions, but others are descriptions of the event.  Determine how to select only the captions.

In [673]:
from lxml import html
import requests
import spacy
nlp = spacy.load("en_core_web_sm")
url = ('https://web.archive.org/web/20151114014941/http://www.newyorksocialdiary.com/party-pictures/2015/celebrating-the-neighborhood')
page = requests.get(url)
soup = BeautifulSoup(page.text, "lxml")
caption = soup.find_all('div', attrs={'class': 'photocaption'})
doc = nlp(str(caption[0]).split(">")[1].split("<")[0])

for ent in doc.ents:
    print(ent.text)

Glenn Adamson
Simon Doonan
Victoire de Castellane
Craig Leavitt
Jerome Chazen
Andi Potamkin
Ralph Pucci
Kirsten Bailey
Edwin Hathaway
Dennis Freedman
the Museum of Art and Design's


In [ ]:
captions = ...

In [638]:
sr1 = []
for ent in doc.ents:
    sr1.append(ent.text)
sr1

['Doug Steinbrech', 'Mary Van']

Let's encapsulate this in a function.  As with the links pages, we want to avoid downloading a given page the next time we need to run the notebook.  While we could save the files by hand, as we did before, a checkpoint library like [ediblepickle](https://pypi.python.org/pypi/ediblepickle/1.1.3) can handle this for you.  (Note, though, that you may not want to enable this until you are sure that your function is working.)

You should also keep in mind that HTTP requests fail occasionally, for transient reasons.  You should plan how to detect and react to these failures.   The [retrying module](https://pypi.python.org/pypi/retrying) is one way to deal with this.

Now that we have some sample captions, let's start parsing names out of those captions.  There are many ways of going about this, and we leave the details up to you.  Some issues to consider:

  1. Some captions are not useful: they contain long narrative texts that explain the event.  Try to find some heuristic rules to separate captions that are a list of names from those that are not.  A few heuristics include:
    - look for sentences (which have verbs) and as opposed to lists of nouns. For example, [`nltk` does part of speech tagging](http://www.nltk.org/book/ch05.html) but it is a little slow. There may also be heuristics that accomplish the same thing.
    - Similarly, spaCy's [entity recognition](https://spacy.io/docs/usage/entity-recognition) could be useful here.
    - Look for commonly repeated threads (e.g. you might end up picking up the photo credits or people such as "a friend").
    - Long captions are often not lists of people.  The cutoff is subjective, but for grading purposes, *set that cutoff at 250 characters*.
  2. You will want to separate the captions based on various forms of punctuation.  Try using `re.split`, which is more sophisticated than `string.split`. **Note**: The reference solution uses regex exclusively for name parsing.
  3. You might find a person named "ra Lebenthal".  There is no one by this name.  Can anyone spot what's happening here?
  4. This site is pretty formal and likes to say things like "Mayor Michael Bloomberg" after his election but "Michael Bloomberg" before his election.  Can you find other ('optional') titles that are being used?  They should probably be filtered out because they ultimately refer to the same person: "Michael Bloomberg."
  5. There is a special case you might find where couples are written as eg. "John and Mary Smith". You will need to write some extra logic to make sure this properly parses to two names: "John Smith" and "Mary Smith".
  6. When parsing names from captions, it can help to look at your output frequently and address the problems that you see coming up, iterating until you have a list that looks reasonable. This is the approach used in the reference solution. Because we can only asymptotically approach perfect identification and entity matching, we have to stop somewhere.


In [1648]:
import requests
import spacy
nlp = spacy.load("en_core_web_sm")
from spacy import displacy
def get_captions(path):
    xxxx
    
    return caption_str
        
url = ('https://web.archive.org/web/20151114014941/http://www.newyorksocialdiary.com/party-pictures/2015/celebrating-the-neighborhood')
captions = get_captions(url)
names = []
pairs = []
for i in range(len(captions)):
    p1 = []  
    n1 = []
    n1.append( find_word(captions[i].replace('\n','')))
    xxx
        pairs.append(p1)

names_sorted = sorted(names)             
names      

['Glenn Adamson',
 'Simon Doonan',
 'Victoire De Castellane',
 'Craig Leavitt',
 'Jerome Chazen',
 'Andi Potamkin',
 'Ralph Pucci',
 'Kirsten Bailey',
 'Edwin Hathaway',
 'Dennis Freedman',
 'Randy Takian',
 'Kamie Lightburn',
 'Christopher Spitzmiller',
 'Diana Quasha',
 'Mariam Azarm',
 'Sana Sabbagh',
 'Lynette Dallas',
 'Sydney Shuman',
 'Matthew Bees',
 'Tom Edelman',
 'Warren Scharf',
 'Amory Mc',
 'Sean Mc',
 'Mario Buatta',
 'Helene Tilney',
 'Katherine De',
 'Elijah Duckworth',
 'John Rosselli',
 'Elizabeth Swartz',
 'Stephen Simcock',
 'Lee Strock',
 'Thomas Hammer',
 'Searcy Dryden',
 'Lesley Dryden',
 'Richard Lightburn',
 'Michel Witmer',
 'Jennifer Cacioppo',
 'Kevin Michael Barba',
 'Virginia Wilbanks',
 'Lacary Sharpe',
 'Valentin Hernandez',
 'Yaz Hernandez',
 'Chele Farley',
 'James Farley',
 'Harry Heissmann',
 'Angela Clofine',
 'Michael Clofine',
 'Jared Goss',
 'Kristina Stewart Ward',
 'Alex Papachristidis',
 'Nick Olsen',
 'Lindsey Coral Harper',
 'Alberto Villa

## Question 2: sample_names


Once you feel that your algorithm is working well on these captions, parse all of the captions and extract all the names mentioned.  Sort them alphabetically, by first name, and return the first hundred.

In [1618]:
sample_names = ["Caroline Dean"] * 100

grader.score('graph__sample_names', names_sorted[0:100])

Your score:  0.9400000000000006


Now, run this sort of test on a few other pages.  You will probably find that other pages have a slightly different HTML structure, as well as new captions that trip up your caption parser.  But don't worry if the parser isn't perfect -- just try to get the easy cases.

## Phase Three


Once you are satisfied that your caption scraper and parser are working, run this for all of the pages.  If you haven't implemented some caching of the captions, you probably want to do this first.

In [11]:
len(Captions)

100329

In [23]:
names

['Les Lieberman',
 'Barri Lieberman',
 'Isabel Kallman',
 'Trish Iervolino',
 'Ron Iervolino',
 'Chuck Grodin',
 'Diana Rosario',
 'Ali Sussman',
 'Sarah Boll',
 'Jen Zaleski',
 'Alysse Brennan',
 'Lindsay Macbeth',
 'Kelly Murro',
 'Tom Murro',
 'Udo Spreitzenbarth',
 'Russ Middleton',
 'Lisa Middleton',
 'Barbara Loughlin',
 'Gerald Loughlin',
 'Debbie Gelston',
 'Julianne Michelle',
 'Heather Robinson',
 'Kiwan Nichols',
 'Jimmy Nichols',
 'Melanie Carbone',
 'Nancy Brown',
 'Bill Mack',
 'David Lyden',
 'Patricia Sorenson',
 'Jimmy Cayne',
 'Vince Tese',
 'Pat Cayne',
 'Stuart Oran',
 'Hilary Oran',
 'Dwight Gooden',
 'Amy Cunningham',
 'Ray Mirra',
 'Tyler Janovitz',
 'Dan Shedrick',
 'Samara Heafitz',
 'Cass Adelman',
 'Jason Adelman',
 'Bart Scott',
 'Mark Laplander',
 'Mitch Rubin',
 'Audra Zuckerman',
 'Michelle Smith',
 'Kenneth Mehlman',
 'Julia Harquail',
 'John Hackett',
 'Judy Poller',
 'Rob Affuso',
 'Angela Bergeson',
 'David Byrnes',
 'Concetta Bencivenga',
 'Nicole Di

In [20]:
len(pairs)

100197

## For the remaining analysis, we think of the problem in terms of a
[network](http://en.wikipedia.org/wiki/Computer_network) or a
[graph](https://en.wikipedia.org/wiki/Graph_%28discrete_mathematics%29).  Any time a pair of people appear in a photo together, that is considered a link.  What we have described is more appropriately called an (undirected)
[multigraph](http://en.wikipedia.org/wiki/Multigraph) with no self-loops but this has an obvious analog in terms of an undirected [weighted graph](http://en.wikipedia.org/wiki/Graph_%28mathematics%29#Weighted_graph).  In this problem, we will analyze the social graph of the New York social elite.  We recommend using python's [`networkx`](https://networkx.github.io/) library.

In [155]:
name_combo2

[[('Les Lieberman', 'Barri Lieberman'),
  ('Les Lieberman', 'Isabel Kallman'),
  ('Les Lieberman', 'Trish Iervolino'),
  ('Les Lieberman', 'Ron Iervolino'),
  ('Barri Lieberman', 'Isabel Kallman'),
  ('Barri Lieberman', 'Trish Iervolino'),
  ('Barri Lieberman', 'Ron Iervolino'),
  ('Isabel Kallman', 'Trish Iervolino'),
  ('Isabel Kallman', 'Ron Iervolino'),
  ('Trish Iervolino', 'Ron Iervolino')],
 [],
 [('Diana Rosario', 'Ali Sussman'),
  ('Diana Rosario', 'Sarah Boll'),
  ('Diana Rosario', 'Jen Zaleski'),
  ('Diana Rosario', 'Alysse Brennan'),
  ('Diana Rosario', 'Lindsay Macbeth'),
  ('Ali Sussman', 'Sarah Boll'),
  ('Ali Sussman', 'Jen Zaleski'),
  ('Ali Sussman', 'Alysse Brennan'),
  ('Ali Sussman', 'Lindsay Macbeth'),
  ('Sarah Boll', 'Jen Zaleski'),
  ('Sarah Boll', 'Alysse Brennan'),
  ('Sarah Boll', 'Lindsay Macbeth'),
  ('Jen Zaleski', 'Alysse Brennan'),
  ('Jen Zaleski', 'Lindsay Macbeth'),
  ('Alysse Brennan', 'Lindsay Macbeth')],
 [('Kelly Murro', 'Tom Murro')],
 [],
 [('R

In [158]:
name_combo2[0][0][0]

'Les Lieberman'

## Question 3: degree


The simplest question to ask is "who is the most popular"?  The easiest way to answer this question is to look at how many connections everyone has.  Return the top 100 people and their degree.  Remember that if an edge of the graph has weight 2, it counts for 2 in the degree.

**Checkpoint:** Some aggregate stats on the solution

    "count": 100.0
    "mean": 189.92
    "std": 87.8053034454
    "min": 124.0
    "25%": 138.0
    "50%": 157.0
    "75%": 195.0
    "max": 666.0

In [125]:
import heapq  # Heaps are efficient structures for tracking the largest
              # elements in a collection.  Use introspection to find the
              # function you need.
degree = [('Alec Baldwin', 144)] * 100

grader.score('graph__degree', pop_names_rev[1:101])

Your score:  0.8900000000000006


In [135]:
pgsorted.pop(0)

[('Jean Shafiroff', 0.0006570968794138355),
 ('Mark Gilbertson', 0.0005037165468333382),
 ('Gillian Miniter', 0.00045589210840444466),
 ('Patrick Mc', 0.0004039345230306752),
 ('Geoffrey Bradfield', 0.00038902405293406205),
 ('Alexandra Lebenthal', 0.00035358667607624634),
 ('Andrew Saffir', 0.0003391638429547283),
 ('Yaz Hernandez', 0.0003063653222373015),
 ('Mario Buatta', 0.0002971678062559083),
 ('Debbie Bancroft', 0.00029400732697882877),
 ('Somers Farkas', 0.0002933975852500674),
 ('Sharon Bush', 0.0002893937986861579),
 ('Michael Bloomberg', 0.000287380730920405),
 ('Kamie Lightburn', 0.00027213478179771955),
 ('Alina Cho', 0.0002637077814900352),
 ('Barbara Tober', 0.0002586142383006657),
 ('Eleanora Kennedy', 0.0002495361830745748),
 ('Lucia Hwong Gordon', 0.0002463142864312178),
 ('Lydia Fenet', 0.00023837989607443022),
 ('Muffie Potter Aston', 0.00023007979067639974),
 ('John Mc', 0.00022599053817799227),
 ('Bonnie Comley', 0.00022476983055593624),
 ('Christopher Hyland', 0.

## Question 4: PageRank


A similar way to determine popularity is to look at their
[PageRank](http://en.wikipedia.org/wiki/PageRank).  PageRank is used for web ranking and was originally
[patented](http://patft.uspto.gov/netacgi/nph-Parser?patentnumber=6285999) by Google and is essentially the stationary distribution of a [Markov
chain](http://en.wikipedia.org/wiki/Markov_chain) implied by the social graph. You can implement this yourself or use the version in `networkx`

Use 0.85 as the damping parameter so that there is a 15% chance of jumping to another vertex at random.

**Checkpoint:** Some aggregate stats on the solution

    "count": 100.0
    "mean": 0.0001841088
    "std": 0.0000758068
    "min": 0.0001238355
    "25%": 0.0001415028
    "50%": 0.0001616183
    "75%": 0.0001972663
    "max": 0.0006085816

In [136]:
pagerank = [('Martha Stewart', 0.00019312108706213307)] * 100

grader.score('graph__pagerank', pg[0:100])

Your score:  0.9200000000000006


## Question 5: best_friends


Another interesting question is who tend to co-occur with each other.  Give us the 100 edges with the highest weights.

Google these people and see what their connection is.  Can we use this to detect instances of infidelity?

**Checkpoint:** Some aggregate stats on the solution

    "count": 100.0
    "mean": 25.84
    "std": 16.0395470855
    "min": 14.0
    "25%": 16.0
    "50%": 19.0
    "75%": 29.25
    "max": 109.0

In [106]:
best_friends = [(('Michael Kennedy', 'Eleanora Kennedy'), 41)] * 100

grader.score('graph__best_friends', edges_sorted[:100])

Your score:  0.9200000000000006


*Copyright &copy; 2020 Pragmatic Institute. This content is licensed solely for personal use. Redistribution or publication of this material is strictly prohibited.*